In [1]:
import os
import sys
import torch
import altair as alt
import pandas as pd
import numpy as np

alt.data_transformers.disable_max_rows();


In [2]:
## TODO: change this to the dump path.
# data_dump_path = '../../swag-3body-data.pt'
data_dump_path = '../../de-3body-data.pt'

assert os.path.isfile(data_dump_path)

data_dump = torch.load(data_dump_path)

ts = data_dump.get('ts')
z0_orig = data_dump.get('z0_orig')
true_zt = data_dump.get('true_zt')
# true_zt_chaos = data_dump.get('true_zt_chaos').permute(1, 0, 2, 3, 4, 5)
de_pred_zt = data_dump.get('pred_zt')

data_dump_path = '../../swag-3body-data.pt'
data_dump = torch.load(data_dump_path)
swag_pred_zt = data_dump.get('pred_zt')

In [14]:
def compute_rel_error(ref, pred):
    '''
    N is the number of initial conditions.
    M is the number of samples in prediction
    The first dimension "2" corresponds to position + velocity.
    B is the number of bodies.
    The last dimension "2" corresponds to xy.

    Arguments:
    ref: N x T x 2 x B x 2
    pred: M x N x T x 2 x B x 2
    '''
    delta_z = ref - pred  # M x N x T x 2 x B x 2
    all_err = delta_z.pow(2).sum(dim=-1).sum(dim=-1).sum(dim=-1).sqrt()  # M x N x T
    return all_err

In [34]:
det_rel_err = compute_rel_error(true_zt, de_pred_zt[1])
de_rel_err = compute_rel_error(true_zt, de_pred_zt.mean(0))
swag_rel_err = compute_rel_error(true_zt, swag_pred_zt.mean(0))
print(de_rel_err.shape)

torch.Size([5, 334])


In [86]:
def plot_rel_err(rel_err, color):
    y_mean = rel_err.mean(0).cpu().numpy()
    y_std = rel_err.std(0).cpu().numpy()

    err_mean_chart = alt.Chart(pd.DataFrame({
        't': ts.cpu().numpy(),
        'y': y_mean,
        'y_hi': y_mean + 1. * y_std,
        'y_lo': np.clip(y_mean - 1. * y_std, 0.0, np.inf),
    })).mark_line(color=color, opacity=0.7).encode(x='t', y='y')
    
    err_std_chart = err_mean_chart.mark_area(color=color, opacity=0.2).encode(
        x='t', y=alt.Y('y_lo', title='RMSE'), y2='y_hi')

    return err_mean_chart + err_std_chart

In [91]:
plot_rel_err(de_rel_err, "green") + plot_rel_err(swag_rel_err, "red") + plot_rel_err(det_rel_err, "blue")

alt.LayerChart(...)